This is a starter notebook, you'll have to import the libraries available in the requirements.txt file in this directory. 

In [ ]:
import os
import json
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

# 🌍 Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# ✨ Initialize LLM and embedding model
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.8)
embedding_model = OpenAIEmbeddings()

# 🏠 Step 1: Generate Real Estate Listings
prompt_template = """
Generate a realistic real estate listing in the following format:
Neighborhood: <neighborhood name>
Price: <$xxx,xxx>
Bedrooms: <int>
Bathrooms: <int>
House Size: <sqft>

Description: <A paragraph with vivid and enticing details about the house>

Neighborhood Description: <A paragraph that describes the neighborhood and nearby amenities>
"""

listings = [llm(prompt_template).strip() for _ in range(10)]

with open("listings.json", "w") as f:
    json.dump(listings, f, indent=2)

# 🧠 Step 2: Store Listings in ChromaDB
os.makedirs("chromadb", exist_ok=True)
db = Chroma(
    persist_directory="chromadb",
    collection_name="real_estate",
    embedding_function=embedding_model
)

texts = listings
metadatas = [{"id": i} for i in range(len(listings))]
db.add_texts(texts, metadatas=metadatas)
db.persist()

# 🙋 Step 3: Collect Buyer Preferences
answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

buyer_profile = " ".join(answers)

# 🔍 Step 4: Semantic Search
results = db.similarity_search(buyer_profile, k=3)

# 🎨 Step 5: Personalized Listings
personalized = []
for res in results:
    prompt = f"""
You are a helpful real estate assistant.

Buyer's Preferences:
{buyer_profile}

Original Listing:
{res.page_content}

Rewrite the listing description and neighborhood description to subtly emphasize aspects aligned with the buyer's preferences, without changing facts.
"""
    personalized.append(llm(prompt).strip())

# 🖼️ Stand-Out Suggestion: Image Search with CLIP
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Dummy image-text dataset (replace with your actual images)
image_folder = "images"  # Folder with images for listings
image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder) if fname.endswith(".jpg")]

image_embeddings = []
for img_path in image_paths:
    image = Image.open(img_path)
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        image_emb = clip_model.get_image_features(**inputs)
        image_embeddings.append((img_path, image_emb))

# Convert buyer profile to image embedding
text_inputs = clip_processor(text=buyer_profile, return_tensors="pt", padding=True)
with torch.no_grad():
    text_embedding = clip_model.get_text_features(**text_inputs)

# Find best matching image
similarities = [(path, torch.nn.functional.cosine_similarity(text_embedding, img_emb).item())
                for path, img_emb in image_embeddings]
similarities.sort(key=lambda x: x[1], reverse=True)

print("\n🎯 Top Matching Image:", similarities[0][0])

# 💾 Save personalized output
with open("personalized_listings.txt", "w") as f:
    for i, listing in enumerate(personalized):
        f.write(f"\n=== Personalized Listing {i+1} ===\n{listing}\n")